Prep data for the LSTM:
1. all data


In [1]:
## using the neuralhydrology package
## 1: prep the data according to the nh standard
import pandas as pd
import xarray as xr
from fos import util
from fos.data import snotelmeta 
import datetime
import os
import numpy as np
from fos.util import get_wrf_data_points

## 
def wrfread(datadir, gcm, exp, variant, domain, var):
    modeldir = datadir + gcm + '/postprocess/'+domain + '/'
    all_files = sorted(os.listdir(modeldir))
    read_files = []
    for ii in all_files:
        if (
            ii.startswith(var + ".")
            and model in ii
            and variant in ii
            and domain in ii
        ):
            if domain in ii:
                read_files.append(os.path.join(modeldir, str(ii)))
    assert len(read_files) > 0, f"No matching files found in {modeldir}"

    del all_files

    data = xr.open_mfdataset(read_files, combine="by_coords")
    var_read = data.variables[var]

    dates = []
    for val in data["day"].data:
        try:
            dates.append(datetime.datetime.strptime(str(val)[0:-2], "%Y%m%d").date())
        except ValueError:
            dates.append(datetime.datetime(int(str(val)[0:4]), int(str(val)[4:6]), 28))


    var_read = xr.DataArray(var_read, dims=["day", "lat2d", "lon2d"])
    var_read["day"] = dates
    return var_read

## load data  
var = 'snow'
modeldir = '/glade/campaign/uwyo/wyom0112/postprocess/'
model = 'ukesm1-0-ll'
variant = 'r2i1p1f2'
domain = 'd02'
basedir = '/glade/u/home/mcowherd/'
projectdir = basedir + 'fos-data/'
snoteldir = projectdir + 'snoteldata/'
wrfdir = '/glade/campaign/uwyo/wyom0112/postprocess/'
wrfcoorddir = projectdir 
domain = "d02"

## LOAD DATA ## 
mod_historical = model +'_'+ variant + '_historical_bc'
mod_future = model +'_' + variant+ '_ssp370_bc'
gcm = mod_historical
date_start_pd, date_end_pd = [1980, 1, 1], [2013, 12, 31]  # 30 years, historical
exp = "hist"
var_wrf = wrfread(modeldir, gcm, exp, variant, domain, var)
var_wrf = util.screen_times_wrf(var_wrf, date_start_pd, date_end_pd)

# future dates
date_start_pd, date_end_pd = [2014, 1, 1], [2100, 12, 31]
gcm = mod_future
model = "ssp370"
var_wrf_ssp370 = wrfread(modeldir, gcm, model, variant, domain, var)
var_wrf_ssp370 = util.screen_times_wrf(var_wrf_ssp370, date_start_pd, date_end_pd)

wrfdata = [var_wrf, var_wrf_ssp370]

coords = util.get_coords(wrfcoorddir+'/wrf_coordinates')

lon_wrf = coords['lon2d']
lat_wrf = coords['lat2d']



wrf_snow_data = []
for i, entry in snotelmeta[100:102].iterrows():
    pt = [entry.lon, entry.lat]
    dis = ((pt[0]-lon_wrf[:].data)**2+(pt[1]-lat_wrf[:].data)**2)**0.5
    ind = np.unravel_index(dis.argmin(), dis.shape)
    j = int(ind[0])
    k = int(ind[1])
    wrfts = get_wrf_data_points(wrfdata, j, k)
    wrf_snow_data.append(wrfts)
    
N = len(wrf_snow_data[0])
index = pd.date_range('1980-01-01', periods=N, freq='D')
data = {'SNOTEL_SWE':wrf_snow_data[0]}
forcing_data = pd.DataFrame(data, index=index)
true_SWE = pd.DataFrame({'SWE': wrf_snow_data[1]}, index=index)


In [ ]:
from fos.models import snotel_value, snotel_with_offset, training_mean, lin_reg
from fos.util import partition_dataframe, make_time_lists
mods_and_params = [(snotel_value, {'vars': ['SWE']}), 
                   (snotel_with_offset,  {'vars': ['SWE']}),
                   (training_mean,  {'vars': ['SWE']}),
                   (lin_reg,  {'vars': ['SWE'], 'fvars': ['SNOTEL_SWE']}),]

# Define start and stop dates for three time periods
time_periods = {'train': ('1980-01-01','2000-01-01'),
           'validation': ('2000-01-01', '2020-01-01'),
           'test': ('2020-01-01','2090-01-01')}


forcing_split = partition_dataframe(forcing_data, time_periods)
obs_split = partition_dataframe(true_SWE, time_periods)
dates_lists = make_time_lists(time_periods)


num = 123456789 ## get it from the thing
datadir = '/glade/u/home/mcowherd/nh-WRF/'


forcing_head = ['Year','Month','Day','Hr', 'SNOTEL_SWE']
## usgs txt fmt = 123456789 1980 01 01 VALUE ## 
forcing_dir = datadir + '/basin_mean_forcing/nldas/'
sf_dir = datadir + 'usgs_streamflow/'
forcing_of = forcing_dir + f'{num}_forcing.txt'
sf_of = sf_dir + f'{num}_streamflow.txt'
nc_of = datadir + f'time_series/{num}.nc'

df = forcing_data.copy()
# Save DataFrame to tab-delimited text file with year, month, date, and value in separate columns
df.reset_index(inplace=True)  # Reset index to use date as a column
df.rename(columns={'index': 'date'}, inplace=True)  # Rename index column to date
df['Year'] = df['date'].dt.year  # Extract year from date
df['Mnth'] = df['date'].dt.month  # Extract month from date
df['Day'] = df['date'].dt.day  # Extract day from date
df = df[['Year', 'Mnth', 'Day', 'SNOTEL_SWE']]  # Reorder columns
df.to_csv(forcing_of, sep='\t', index=False)  # Save DataFrame to tab-delimited text file

df = true_SWE.copy()
df.reset_index(inplace=True)  # Reset index to use date as a column
df.rename(columns={'index': 'date'}, inplace=True)  # Rename index column to da
basin = [num for i in range(len(df))]
df['basin'] = basin
df['Year'] = df['date'].dt.year  # Extract year from date
df['Mnth'] = df['date'].dt.month  # Extract month from date
df['Day'] = df['date'].dt.day  # Extract day from date
df = df[['basin', 'Year', 'Mnth', 'Day', 'SWE']]  # Reorder columns
df.to_csv(sf_of, sep='\t',header = False, index=False)  # Save DataFrame to tab-delimited text file



# Read in tab-delimited text file and create DataFrame
df = pd.read_csv(forcing_of, sep='\t', header=0, names=['year', 'month', 'day', 'SWE'])
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df.set_index('date')
df = df.drop(['year', 'month', 'day'], axis=1)

true_df = pd.read_csv(sf_of, sep='\t', header=0, names=['year', 'month', 'day', 'SNOTEL_SWE'])
true_df['date'] = pd.to_datetime(true_df[['year', 'month', 'day']])
true_df = true_df.set_index('date')
true_df = true_df.drop(['year', 'month', 'day'], axis=1)

df = pd.merge(df, true_df, on=['date'])
# Convert DataFrame to xarray Dataset and create 'date' coordinate from year, month, and day columns
ds = xr.Dataset.from_dataframe(df)

# Save xarray Dataset to netCDF file
ds.to_netcdf(nc_of)


prep data for LSTM:
    all points, just peak data

In [6]:
dpath = "data/ukesm"
plotpath = "plots/dev"

from fos.util import setup_plot_style
from fos.util import memory

os.makedirs(plotpath, exist_ok=True)

In [7]:
## HELPERS
def read_csvs(dpath: str) -> pd.DataFrame:
  dfs = list()
  for filename in dpath:
      data = pd.read_csv(filename, index_col=None, header=0)
      data['pt'] = filename.stem.split('_')[-1]
      dfs.append(data)
  df = pd.concat(dfs, axis=0, ignore_index=True)
  df = df.rename(columns={'Unnamed: 0': 'wyear'})
  return df


@memory.cache
def get_data(dpath: str) -> pd.DataFrame:  
  basins = read_csvs(Path(dpath).glob("wrfb*.csv"))
  pts = read_csvs(Path(dpath).glob("wrfp*.csv"))
  combined_df = pd.merge(pts, basins, how='left', left_on=['pt', 'wyear'], right_on=['pt', 'wyear'], suffixes=('_pt', '_basin'))
  return combined_df


def savefig(name: str):
  plt.savefig(os.path.join(plotpath, name))


def plt_err_hist(dffield, xlabel, ylabel, title, fname):
  plt.clf()
  sns.histplot(dffield)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.title(title)
  savefig(fname)
  plt.show()

def plt_year_err(data, title, xlab, ylab, fname):
  plt.clf()
  mean_data = data.mean()
  std_data = data.std()
  mean_data.plot()
  plt.fill_between(mean_data.index, mean_data - std_data, mean_data +  std_data, color="b", alpha=0.2);
  plt.ylabel(ylab)
  plt.xlabel(xlab)
  plt.title(title)
  savefig(fname)
  plt.show()
  
def plot_relation(df, x, y, title, fname, c='wyear', colormap='viridis'):
  plt.clf()
  df.plot.scatter(x=x, y=y, c=c, colormap=colormap, title=title)
  plt.gca().axline((0, 0), slope=1)
  savefig(fname)
  plt.show()


In [8]:
# read in the data from dpath
df = get_data(dpath)
df.head()

orig_len = len(df)
# clean the data
df.dropna(inplace=True)
new_len = len(df)
print(f"dropped {orig_len - new_len}/{orig_len} rows with NaNs")


dropped 5616/96408 rows with NaNs


In [12]:
from fos.data import huc6, in6s

In [13]:
## goal: save these as nc files
## one nc file per basin with name nc_of = datadir + f'time_series/{num}.nc'
## date index
## in each file, all the point values for the poi# and possibly all the other derived variables once we get to that point? 
## then can try different version of each -- combinations of all of the different variables
## utkarsh for predictors
## shiheng for methods
## earlier stuff (multivariable regression) for argument that it will decay
basin_ids = np.unique(in6s) ## 91 total ? 

In [ ]:

from fos.dirs import basedir, projectdir, snoteldir
import geopandas as gpd
import pandas as pd

'''
snotelmeta
snotel_gdf
in6s, in8s
snotel_no_ak
'''

snotelmeta = pd.read_csv(snoteldir + 'snotelmeta.csv')
huc6 = gpd.read_file(projectdir + 'spatialdata/huc6.shp')
huc8 = gpd.read_file(projectdir + 'spatialdata/huc8.shp')
snotel_gdf = gpd.GeoDataFrame(data = {'site_name':snotelmeta.site_name,
                                     'elev': snotelmeta.elev,
                                     'site_number':snotelmeta.site_number,
                                     'state':snotelmeta.state,
                                     'namestr':snotelmeta.namestr,
                                     'startdt':snotelmeta.startdt}, geometry = gpd.points_from_xy(snotelmeta.lon, snotelmeta.lat), crs = 'epsg:4326')

## time series of the difference in time between peak SWE at the grid box where the SNOTEL resides 
## and the time of peak SWE across the HUC-6 watershed in which the SNOTEL resides
in6 = []
in8 = []
ptnames = []
for i in snotel_gdf.index:
    point = snotel_gdf[snotel_gdf.index == i].geometry[i]
    ptname = snotel_gdf[snotel_gdf.index == i].site_number[i]
    basin = huc6[huc6.contains(point)]
    k = basin.name.index[0]
    in6.append(basin.name[k])
    basin = huc8[huc8.contains(point)]
    k = basin.name.index[0]
    in8.append(basin.name[k])
    ptnames.append(ptname)
in6s = pd.Series(in6, index = ptnames)
in8s = pd.Series(in8, index = ptnames)

In [ ]:
in6s